In [10]:
%%time

import os
import boto3
import re
import json
import sagemaker
from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-xgboost-byo"
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)


CPU times: user 101 ms, sys: 3.93 ms, total: 105 ms
Wall time: 288 ms


In [2]:
!conda install -y -c conda-forge xgboost==0.90


Solving environment: done

# All requested packages already installed.



In [3]:
!aws s3 ls s3://sagemaker-sample-files/datasets/image/MNIST/


                           PRE MNIST/
                           PRE model/
                           PRE pytorch/
                           PRE test/
                           PRE train/
2020-11-18 02:38:22          0 
2020-12-28 19:34:02  220080426 mnist.pkl
2021-05-20 17:34:52   16168813 mnist.pkl.gz
2020-11-18 02:41:35    1648877 t10k-images-idx3-ubyte.gz
2020-11-18 02:41:27       4542 t10k-labels-idx1-ubyte.gz
2020-11-18 02:41:45    9912422 train-images-idx3-ubyte.gz
2020-11-18 02:41:28      28881 train-labels-idx1-ubyte.gz


In [4]:
%%time
import pickle
import boto3
import gzip

# Get the data from a public S3
buf=boto3.client('s3').get_object(
    Bucket='sagemaker-sample-files',
    Key='datasets/image/MNIST/mnist.pkl.gz'
)['Body'].read()


# decompress the buffer
decomp_buf = gzip.decompress(buf)
train_set, valid_set, test_set = pickle.loads(decomp_buf, encoding="latin1")


CPU times: user 906 ms, sys: 450 ms, total: 1.36 s
Wall time: 2.68 s


In [5]:
train_X = train_set[0]
train_y = train_set[1]

valid_X = valid_set[0]
valid_y = valid_set[1]

test_X = test_set[0]
test_y = test_set[1]


In [6]:
import xgboost as xgb
import sklearn as sk

bt = xgb.XGBClassifier(
    max_depth=5, learning_rate=0.2, n_estimators=10, objective="multi:softmax"
)  # Setup xgboost model
bt.fit(train_X, train_y, eval_set=[(valid_X, valid_y)], verbose=False)  # Train it to our data


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [11]:
model_file_name = "DEMO-local-xgboost-model"
bt.save_model(model_file_name)

In [12]:
!tar czvf model.tar.gz $model_file_name


DEMO-local-xgboost-model


In [13]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")


boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)


In [14]:
from sagemaker.amazon.amazon_estimator import get_image_uri

container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-2")


The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [15]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)
sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])


https://s3-eu-west-1.amazonaws.com/sagemaker-eu-west-1-982111424575/sagemaker/DEMO-xgboost-byo/DEMO-local-xgboost-model/model.tar.gz
arn:aws:sagemaker:eu-west-1:982111424575:model/demo-local-xgboost-model2022-02-01-12-38-38
CPU times: user 16.7 ms, sys: 0 ns, total: 16.7 ms
Wall time: 381 ms


In [16]:
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoostServerlessEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "ServerlessConfig": {
                "MemorySizeInMB": 2048,
                "MaxConcurrency": 20
            },
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])


DEMO-XGBoostServerlessEndpointConfig-2022-02-01-12-38-46
Endpoint Config Arn: arn:aws:sagemaker:eu-west-1:982111424575:endpoint-config/demo-xgboostserverlessendpointconfig-2022-02-01-12-38-46


In [ ]:
%%time
import time

endpoint_name = "DEMO-XGBoostServerlessEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)


DEMO-XGBoostServerlessEndpoint-2022-02-01-12-38-55
arn:aws:sagemaker:eu-west-1:982111424575:endpoint/demo-xgboostserverlessendpoint-2022-02-01-12-38-55
Status: Creating
Status: Creating


In [ ]:
runtime_client = boto3.client("runtime.sagemaker")

In [ ]:
import numpy as np

point_X = test_X[0]
point_X = np.expand_dims(point_X, axis=0)
point_y = test_y[0]
np.savetxt("test_point.csv", point_X, delimiter=",")


In [ ]:
%%time
import json


file_name = (
    "test_point.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)
result = response["Body"].read().decode("ascii")
print("Predicted Class Probabilities: {}.".format(result))


In [ ]:
floatArr = np.array(json.loads(result))
predictedLabel = np.argmax(floatArr)
print("Predicted Class Label: {}.".format(predictedLabel))
print("Actual Class Label: {}.".format(point_y))
